Federated learning on sequence models using LSTM
===================================================

Based on https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import syft as sy
hook = sy.TorchHook(torch)
#Welcome bob&alice
bob = sy.VirtualWorker(hook, id="bob") 
alice = sy.VirtualWorker(hook, id="alice") 

In [4]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

## 1. Convert words to indexs, suppose we have a global dictionary of all words
#word_to_ix
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

#tag_to_ix
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

##2. Point data to worker

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.float, requires_grad=True)

data_bob = prepare_sequence(training_data[0][0], word_to_ix)
target_bob = prepare_sequence(training_data[0][1], tag_to_ix)


data_alice = prepare_sequence(training_data[1][0], word_to_ix)
target_alice = prepare_sequence(training_data[1][1], tag_to_ix)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [5]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [6]:
print(datasets)

[((Wrapper)>[PointerTensor | me:11783449835 -> bob:98778365263], (Wrapper)>[PointerTensor | me:41075836959 -> bob:13757575109]), ((Wrapper)>[PointerTensor | me:32355958392 -> alice:93076476785], (Wrapper)>[PointerTensor | me:90749115209 -> alice:10532165511])]


In [6]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


for epoch in range(10): 
    for data, targets in datasets:
        model.send(data.location)
        
        model.zero_grad()

        model.hidden = model.init_hidden()

        print('sentence_in', data)
        tag_scores = model(data)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        model.get()
        print(loss.get())

sentence_in (Wrapper)>[PointerTensor | me:88828310489 -> bob:95839483168]


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)